## <u>Task 2. Use Bokeh + GeoPandas to plot maps </u>

### 1. Import Libraries

In [1]:
%pip install pyproj

import pandas as pd
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar, Select, Slider
from bokeh.layouts import column, row
from bokeh.palettes import Viridis256
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from pyproj import Transformer

Note: you may need to restart the kernel to use updated packages.


### 2. Function to convert lat/lng to Web Mercator (for Bokeh tiles)

In [2]:
def wgs84_to_web_mercator(lng, lat):
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    x, y = transformer.transform(lng, lat)
    return x, y

### 3. Load the CLEANED data

In [3]:
df = pd.read_csv('CAQI.csv')

### 4. Convert lat/lng to Web Mercator coordinates

In [4]:
df['x'], df['y'] = wgs84_to_web_mercator(df['lng'].values, df['lat'].values)

### 5. Create the main data source for the plot

In [5]:
source = ColumnDataSource(data=df)

### 6. Set up the main plot with a map tile

In [6]:
from bokeh.tile_providers import get_provider

p = figure(
    x_axis_type="mercator",
    y_axis_type="mercator",
    tools="pan,wheel_zoom,box_zoom,reset,save",
    title="Live Air Quality Monitoring",
    width=1000,
    height=600
)
p.add_tile(get_provider(CARTODBPOSITRON))

TileRenderer(id='p1037', ...)

### 7. Create a color mapper based on your PM2.5 values

In [7]:
color_mapper = LinearColorMapper(
    palette=Viridis256,
    low=df['AQI Value'].min(), 
    high=df['AQI Value'].max()
)

### 8. Plot the data points on the map

In [8]:
points = p.circle(
    x='x',
    y='y',
    source=source,
    size=8,
    color={'field': 'AQI Value', 'transform': color_mapper}, 
    alpha=0.7,
    legend_label="AQI Value" 
)

### 9. Add a color bar to show the scale

In [9]:
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0), title='AQI Value')
p.add_layout(color_bar, 'right')

## <u> Task 3. Add interactive elements</u >

### 1. Add Hover Tooltips

In [10]:
hover = HoverTool(
    renderers=[points],
    tooltips=[
        ("City", "@City"),
        ("Country", "@Country"),
        ("AQI Value", "@{AQI Value}"),
        ("AQI Category", "@{AQICategory}"),
        ("PM2.5", "@{PM2.5 AQI Value}"),
        ("Ozone", "@{Ozone AQI Value}"),
        ("Timestamp", "@timestamp{%F %T}"),
        ("Loaction", "(@lat, @lng)")
    ],
    formatters={
        '@timestamp': 'datetime'  
    }
)
p.add_tools(hover)
print("Hover tooltips added.")

Hover tooltips added.


### 2. Add a Time Slider

In [11]:
from bokeh.models import CustomJS

df['time_numeric'] = pd.to_datetime(df['timestamp']).astype('int64') // 10**9
time_min = int(df['time_numeric'].min())
time_max = int(df['time_numeric'].max())

time_slider = Slider(start=time_min, end=time_max, value=time_min, 
                    step=3600, title="Time (seconds from start)")  

time_slider.js_on_change('value', CustomJS(args={'source': source}, code="""
    const data = source.data;
    const selected_time = cb_obj.value;
    const time_data = data['time_numeric'];
    const n = time_data.length;
    
    // Create a new array of indices to keep (within ± 1800 seconds = 30 minutes)
    const new_indices = [];
    for (let i = 0; i < n; i++) {
        if (Math.abs(time_data[i] - selected_time) < 1800) {
            new_indices.push(i);
        }
    }
    source.selected.indices = new_indices;
"""))
print("Time slider added.")

Time slider added.


### 3. Add a Filter for Categories

In [12]:
def categorize_aqi(aqi):
    if aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy"
    else:
        return "Hazardous"

df['risk'] = df['AQI Value'].apply(categorize_aqi)
source.data['risk'] = df['risk'] 

risk_options = ['All'] + sorted(df['risk'].unique().tolist())
risk_filter = Select(title="Filter by Risk Level:", value="All", options=risk_options)

risk_filter.js_on_change('value', CustomJS(args={'source': source}, code="""
    const selected_risk = cb_obj.value;
    const risk_data = source.data['risk'];
    const n = risk_data.length;
    
    const new_indices = [];
    for (let i = 0; i < n; i++) {
        if (selected_risk === 'All' || risk_data[i] === selected_risk) {
            new_indices.push(i);
        }
    }
    source.selected.indices = new_indices;
"""))
print("Risk filter added.")

Risk filter added.


* ### Final Layout and Deployment

In [13]:
layout = column(risk_filter, time_slider, p)

curdoc().add_root(layout)
curdoc().title = "Global Air Quality Dashboard"

print("Dashboard setup complete. Running Bokeh server...")

Dashboard setup complete. Running Bokeh server...


## <u>Task 4. Provide choropleth-style visualization</u>

### 1. Create a heatmap effect

In [14]:
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import Plasma256

### 2. Create a more detailed color mapper for the choropleth effect

In [15]:
choropleth_mapper = LinearColorMapper(
    palette=Plasma256,  # Alternative color palette
    low=df['AQI Value'].min(),
    high=df['AQI Value'].max(),
    nan_color='gray'  # Color for missing values
)

### 3. Re-plot the points with the choropleth color mapping through removing previous points firstly

In [16]:
p.renderers = [p.renderers[0]]  # Keep only the base tile


### 4. Add the choropleth-style points

In [17]:
choropleth_points = p.circle(
    x='x',
    y='y',
    source=source,
    size=12, 
    color={'field': 'AQI Value', 'transform': choropleth_mapper},
    alpha=0.8,
    line_color='white',
    line_width=0.5,
    legend_label="AQI Intensity"
)

### 5. Add a proper color bar for the choropleth

In [18]:
color_bar = ColorBar(
    color_mapper=choropleth_mapper,
    label_standoff=15,
    width=20,
    height=400,
    location=(0, 0),
    title="AQI Intensity",
    title_text_font_size='12pt'
)
p.add_layout(color_bar, 'right')

print("Choropleth-style visualization added with intensity-based coloring.")

Choropleth-style visualization added with intensity-based coloring.
